In [45]:
#imports
import pandas as pd
import numpy as np
import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from rake_nltk import Rake
import json
from profanity_check import predict, predict_prob

In [46]:
#load camera dataset
#dataframe = pd.read_csv("camera.csv")
dataframe = pd.read_csv("book.csv")
#dataframe = pd.read_csv("tool.csv")
#dataframe

In [47]:
#sentiment analysis
def get_sentiment(comment):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(comment) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    filtered_sentence = [] 
  
    for w in word_tokens: 
        if w not in stop_words: 
            filtered_sentence.append(w)
    
    sentiment_score = TextBlob(comment).sentiment.polarity
    if(sentiment_score < -0.1):
        sentiment = "negative"
    elif(sentiment_score > 0.1):
        sentiment = "positive"
    else:
        sentiment = "neutral"
    return [sentiment_score, sentiment]

In [48]:
#keyword extraction
def get_keywords(comment):
    r = Rake()
    r.extract_keywords_from_text(comment)
    keywords = r.get_ranked_phrases()
    return keywords[:10]

In [49]:
#vulgarity check
def get_vulgarity(comment):
    v = []
    v.append(comment)
    v = predict_prob(v)
    return v

In [50]:
#create objects
all_data = dataframe.values

data_list = []

for data in all_data:
    obj = {}
    obj["product_id"] = data[3]
    obj["product_name"] = data[5]
    obj["product_category"] = data[6]
    obj["reviewer_id"] = data[1]
    obj["review_date"] = data[14]
    obj["review_header"] = data[12]
    obj["review_body"] = data[13].replace("<br />", '').replace('\\', '')
    obj["star_rating"] = data[7]
    obj["sentiment_score"] = get_sentiment(data[13].replace("<br />", '').replace('\\', ''))[0]
    obj["sentiment"] = get_sentiment(data[13].replace("<br />", '').replace('\\', ''))[1]
    obj["keywords"] = get_keywords(data[13].replace("<br />", '').replace('\\', ''))
    obj["vulgarity"] = get_vulgarity(data[13].replace("<br />", '').replace('\\', ''))[0]
    data_list.append(obj)

#data_list

In [51]:
#save in json
with open('alfred_data_book.json', 'w') as fout:
    json.dump(data_list, fout)